In [40]:
import torch
import torchvision
from torchvision import models
import torchvision.transforms as T
import cv2
import numpy as np
print(torch.__version__)
print(torchvision.__version__)
# colab 환경을 위한 패키지
from google.colab.patches import cv2_imshow

1.7.0+cu101
0.8.1+cu101


In [41]:
# GPU 사용 여부
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [42]:
COLORS = np.array([
    (255, 255, 255), # background -> 하얀색
    (192, 128, 128), # person
 
])

In [43]:
model = models.segmentation.deeplabv3_resnet101(pretrained=True).eval()
if device == 'cuda':
    model = model.to(device)

In [44]:
def preprocess(img):
    trf = T.Compose([
                    T.ToTensor(),
                    # T.CenterCrop(IMG_SIZE),
                    T.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std = [0.229, 0.224, 0.225]
                    ),
    ])
    input_img = trf(img).unsqueeze(0)
    if device == 'cuda':
        input_img = input_img.to(device)
    return input_img

In [57]:
def seg_map(img, n_classes=21):
    color_idx = 0
    rgb = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    for i in (0,15): # 사람과 배경의 경우만 색칠
        idx = img == i
        rgb[idx] = COLORS[color_idx]
        color_idx = color_idx + 1

    return rgb

In [58]:
IMG_SIZE = 480
count = 0
cap = cv2.VideoCapture('./testvid.mp4')
nof_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
ret, img = cap.read()
img.shape

(1080, 1920, 3)

In [59]:
# video writer 구성
video_format = cv2.VideoWriter_fourcc(*'MP4V')
out_video = cv2.VideoWriter('./output.mp4',video_format,cap.get(cv2.CAP_PROP_FPS),(IMG_SIZE,int(img.shape[0] * IMG_SIZE / img.shape[1])))

In [ ]:
while cap.isOpened():
    ret, img = cap.read()
    
    # 이미지 사이즈 조정
    if not ret:
        break;
    print(ret)
    img = cv2.resize(img,(IMG_SIZE,int(img.shape[0] * IMG_SIZE / img.shape[1])))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    input_tensor = preprocess(img)
    out = model(input_tensor)['out'][0]
    # print(out.shape) (21,270,480)
    out = out.argmax(0).detach().cpu().numpy()
    # print(np.unique(out))   # 0  15 배경과 사람만 detect
    out = seg_map(out)
    # cv2_imshow(out)
    out = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
    # print(out.shape)  (270, 480, 3)
    out_video.write(out)
    # print(seg_map(out))
    print("Frame : ", nof_frame, count)
    count = count + 1
cap.release()
out_video.release()